### Initial Setup

The following cells install dependencies and set up LLM tracing.


**Note:** The tracing setup for a notebook is slightly more involved than the setup for an application would be. In most non-notebook settings, you can start tracing simply by calling `ddtrace-run` with the appropriate env vars:

```
DD_LLMOBS_ENABLED=1 DD_LLMOBS_NO_APM=1 DD_LLMOBS_APP_NAME=<YOUR_APP_NAME> DD_API_KEY=<YOUR_DATADOG_API_KEY> DD_SITE=<YOUR_DATADOG_SITE> ddtrace-run python [filename].py
```


In [10]:
# load required env vars for ddtrace
from dotenv import load_dotenv

load_dotenv()

True

In [11]:
import ddtrace.auto

ddtrace.patch_all()

from ddtrace.llmobs import LLMObs

LLMObs.enable()

### Tracing an LLM Span

An LLM Span represents a call to a model. In this example, we are asking `gpt-3.5-turbo` to summarize a provided text and identify a list of topics from the text.

Because we use OpenAI, the call to the LLM is instrumented automatically:


In [12]:
from openai import OpenAI
import os

oai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

sys_prompt = """
Your task is to 
1. Summarize the given text at a 6th grade reading level in no more than 2 sentences.
2. Identify what topics the text belongs to that would allow you to categorize it in a school library.
Format your output strictly following this JSON convention:
{	
    "topics": <[insert array of topics here]>
    "summary": <insert summary here>
}
	"""


def summarize(text, prompt=sys_prompt):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": text},
    ]
    # llm span auto-instrumented via our openai integration
    return (
        oai_client.chat.completions.create(
            messages=messages,
            model="gpt-3.5-turbo",
            response_format={"type": "json_object"},
        )
        .choices[0]
        .message.content
    )

In [13]:
text = """
i truly hate this
"""

In [14]:
import json

# print out the LLM's response:
json.loads(summarize(text))

Error submitting packet: [Errno 61] Connection refused, dropping the packet and closing the socket, 2 additional messages skipped


{'topics': [],
 'summary': 'The text does not provide any information to summarize.'}

Now, try checking out the LLM Observability interace in Datadog. You should see a trace that describes the LLM call, including the system prompt, the user prompt, and the response.
